# Screen

## Data format

We assume input screen data has the following columns:
* `screen_status`: 0=off, 1=on, 2=locked, 3=unlocked

## {func}`niimpy.screen.screen_off`

This returns a dataframe that contains screen off times.  You may think that this is a simple filtering of the screen data, but the phone can turn off while the screen is on.  Thus, the data must be combined with battery data which provides shutdown information.

First we generate the sample dataframes:

In [1]:
import niimpy

df_screen = niimpy.read_csv_string("""\
time,screen_status
0,1
60,0
3600,1
# There is no corresponding screen off event
""")
df_screen

,time,screen_status
1970-01-01 02:00:00+02:00,0,1
1970-01-01 02:01:00+02:00,60,0
1970-01-01 03:00:00+02:00,3600,1


In [2]:
df_battery = niimpy.read_csv_string("""\
time,battery_status
# Battery turns off
3660,-1
""")
df_battery

,time,battery_status
1970-01-01 03:01:00+02:00,3660,-1


Now we can do the calculation, and we see that it returns only the off times.  At time=`03:01:00` there is an off event when the power completely turns off

In [3]:
niimpy.screen_off(df_screen, battery=df_battery)

,screen_status,datetime
1970-01-01 02:01:00+02:00,0.0,1970-01-01 02:01:00+02:00
1970-01-01 03:01:00+02:00,0.0,1970-01-01 03:01:00+02:00


## {func}`niimpy.screen.screen_duration`

This function computes the durations of screen being on and aggregates by day.  Just like with `screen_off` above, it combines with the battery data in order to detect other off events.  It returns

TODO:
- timeout if on for more than two hours.

In [4]:
duration, count = niimpy.screen_duration(df_screen, battery=df_battery)

No missing values


We see that the device has been on for 120 seconds and off for (on hour minus one minute=3540 seconds):
- TODO: this is wrong

In [5]:
duration

group,off,on
datetime,,
1970-01-01 00:00:00+02:00,3540,60


In [6]:
count

group,off_count,on_count
datetime,,
1970-01-01 00:00:00+02:00,1,1
